In [ ]:
import cv2
import numpy as np

In [ ]:
# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels (COCO dataset)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
# Initialize video capture (0 for webcam or provide video file path)
cap = cv2.VideoCapture(0)  # Use a video file path for video detection

In [ ]:
def detect_objects(frame):
    height, width, channels = frame.shape

    # Convert frame to blob and pass through YOLO network
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    
    # Loop through network outputs and collect detection data
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply non-maximum suppression to remove redundant boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    return boxes, confidences, class_ids, indexes

In [ ]:
def draw_labels(frame, boxes, confidences, class_ids, indexes):
    font = cv2.FONT_HERSHEY_SIMPLEX
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[i]

            # Draw bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), font, 0.5, color, 2)
    
    return frame

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects
    boxes, confidences, class_ids, indexes = detect_objects(frame)
    
    # Draw labels and bounding boxes
    frame = draw_labels(frame, boxes, confidences, class_ids, indexes)
    
    # Display the frame
    cv2.imshow("Object Detection", frame)
    
    # Press 'q' to quit
    if cv2.waitKey(1) == ord("q"):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()